# Wrangle OpenStreetMap Data

### Introduction:
* Map Area: Mumbai, Maharashtra, India

* Dataset: This data contains information about the city Mumbai also called the 'city of dreams' since the Indian Movie industry     is located in this city also known as Bollywood. Every year thousands of people come here to either get in the movie industry or music industry. Besides all this there are many tourists spots and old monuments.
I have chosen this dataset since i live in a city nearby called Pune. I wanted to explore what kind of information the Open Street Map data contains about Mumbai since this is an important place so i wanted to contribute to the data by removing any errors if any.

### Problems Encountered in the Map
#### 1. Identifying the problems in the extract

Size of the dataset : 406 MB
Size of sample extract I have created by the code which was given: 41.1 MB


I used two techniques for identifying any problems in the sample dataset.
* Analyzing the sample osm data itself by opening in an editor.
* Modifying the "audit.py" and "process_osm.py" which created the sample osm JSON file and after wards analyzing that data set to search or identify errors if any. 
* Again repeat the steps above.

#### 2.Problems faced

After doing the analysis I found four main problems. These are:

#### a. Over-abbreviated Street Names and Street names in regional language. 

For correcting this I mapped the inconsistent data to correct data. A sample example is shown below.
* "Vasai Station Rd" : "Vasai Station Road"
* "Yashavant Nagar Rd" : "Yashavant Nagar Road"
* "mankoli naka" : "mankoli Toll"

There were also inconsistent street name which made no sense at all and i was not sure what to do so i took them in a lsit and removed those nodes or ways from my analysis.

Invalid street names
 = ["govandi" , "World" , "compound" ,"Powai", "Multiplex", "Gokuldham"]

#### b. Incorrect Phone number format. 

The data has several numbers which are in the wrong format, some of them are given below.

*  '022 2643 5361'
*  '+91 22 2265 4194 / 2263 0393'
*  '+98 22 65 28 52 84'
*  '8108957786'
*  '(91)-9972526110'


First of all the length of a phone number should be 10 and it should be preceded by our country code '+91' for example

* "(91)-9972526110" to "+919972526110"
* "022 2643 5361" to "+912226435361"

#### c. Incorrect cuisine and bank names 

After auditing the data and creating the JSON file. WHen i started to query the data. I found out that the names of these attributes are inconsistent. SO like street name data I mapped them to thier correct description. Below are some sample  mappings I have done.

Cuisine Mapping

* "burger" : "American",
* "pizza" : "Italian",
* "coffee" : "Italian",
* "regional": "Maharashtrian"

Bank Names Mapping

* "Greater" : "Greater Bombay Co-operative Bank",
* "Union" : "Union Bank of India ",
* "State" : "State Bank of India",
* "SBI" : "State Bank of India",


#### d. Name field not there in some of the documents

While analyzing the data I found that some documents did not have the name field in it, instead their name was given in different fields like "operator" or "brand. So I modified the "process_osm.py" to get the name field populated in such cases.

### Auditing the data and filtering out inconsistent information

For this purpose i have written the code below to implement the same.

#### 1.sample_creater.py

Creates a sample extract of the main OSM file so that it is easy to work with the data.

In [3]:
from pprint import pprint
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:/New folder/mumbai_india.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:/New folder/sample_mumbai.osm"


def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(bytes('<?xml version="1.0" encoding="UTF-8"?>\n', 'UTF-8'))
    output.write(bytes('<osm>\n  ', 'UTF-8'))

    # Write every 10th top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % 10 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(bytes('</osm>', 'UTF-8'))

#### 2. audit.py

This code audits the data extract, finds the problems and updates them with proper information.

In [5]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "sample_mumbai.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

unwanted = ["govandi" , "World" , "compound" ,"Powai", "Multiplex", "Gokuldham"]

# UPDATE THIS VARIABLE

#creating mapping for street, cuisines and bank names
mapping = { "St": "Street",
            "St.": "Street",
            "Rd." : "Road",
            "Rd" : "Road",
            "Ave" : "Avenue",
            "stn" : "Station",
            "naka" : "Toll",
            "Naka" : "Toll",
            "Restauran" : "Restaurant"
            }

cuisine_mapping = {"burger" : "American",
                   "pizza" : "Italian",
                   "coffee" : "Italian",
                   "regional": "Maharashtrian",
                   "cake" : "Pastry",
                   "vegetarian":"Indian",
                   "coffee_shop" : "Italian",
                   "bread amlet" : "Indian",
                   "all_types_of_food": "Indian",
                   "indian_aagri": "Indian",
                   "international" : "continental"
    
                   }



bank_mapping = {"Greater" : "Greater Bombay Co-operative Bank",
                "Union" : "Union Bank of India ",
                "State" : "State Bank of India",
                "Dena" : "Dena Bank",
                "HDFC" : "HDFC Bank",
                "Kotak" : "Kotak Mahindra Bank",
                "Central": "Central Bank of India",
                "Karnataka": "Karnataka Bank",
                "HSBC" : "HSBC Bank",
                "SBI" : "State Bank of India",
                "Raheja" : "Raheja Classique Bank",
                "Punjab" : "Punjab National Bank",
                "AXIS" : "AXIS Bank"
               }

#creating a procedure audit_street_type for cleaning the street names

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)            # using regex to get the street name 
   
    if m:
        street_type = m.group()
        if street_type not in expected and not street_type.isdigit() and street_type not in unwanted:
            if ',' not in street_type:
                if street_type == 'Maharashtra':
                    street_types["Road"].add(street_name)
                else:
                    street_types[street_type].add(street_name)
            else:
                pos_type = street_type.find(',')
                pos_name = street_name.find(',')
                
                street_type = street_type[:pos_type]
                street_name = street_name[:pos_name]
                #print (street_name)
                
                street_types[street_type].add(street_name)
    


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, encoding="utf8")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


#defining procedures for cleaning and updating name field, telephone field, cuisines and bank names field.

def update_name(name, mapping):

    # YOUR CODE HERE
    last_word = street_type_re.search(name)
    last_word = (last_word.group(0))
    if (last_word) in (mapping.keys()):
        
        name = name.replace(last_word, mapping[last_word])
        #print(name)
    return name

def update_telephone_no(nos):
    mod_num=''
    if '/' in nos:
                 
        splittednmbers=re.split('/',nos)
        #print(splittednmbers)
        value=[]
        for c in splittednmbers:
            b=''
            
            c=re.split('[^0-9]',c)
            for i in c:
                b = b + i
            if '91' not in b[0:2]:    
                value.append('+9122' + b)
            else:
                value.append('+' + b)
        return(value)
      
    else:
        nos= re.split('[ ()-]',nos)
        for num in nos:
            mod_num = mod_num + num
            
        
        length = len(mod_num)
        rem = length - 10
        mod_num =  '+91' +  mod_num[rem:]
        return (mod_num)
        
def update_food(items):
    
    
    if ';' in items:
        new_cuisine = re.split('[;]',items)
        if 'asian' in new_cuisine or 'french' in new_cuisine:
            return('continental')
        elif 'indian' in new_cuisine:
            return('indian')
        elif 'american' in new_cuisine:
            return('american')
            
    elif items in cuisine_mapping:
        return(cuisine_mapping[items])
    else:
        return(items)
    
    
def update_bank_name(name):
    pos = name.find(" ",1)
    #print(pos)
    if pos != -1:
        first_word = name[:pos]
        if first_word in bank_mapping:
            return bank_mapping[first_word]
        else:
            return name
    elif name in bank_mapping:
        return bank_mapping[name]
    else:
        return name

In [6]:
st_types = audit(OSMFILE)   # calling the audit() prcedure to create the updated name set.

#### 3. process_osm.py

This file is very important since here I am creating the JSON type document for MongoDB plus it also does some small data cleaning also.

In [9]:
import xml.etree.cElementTree as ET
from pprint import pprint
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
    

##the procedure below helps in filtering the fields with proper naming conventions and data. it also creates the JSON object 
##structure for each node and way which we need for our analysis purpose
    
def shape_element(element):
    node = {}
    address={}
    if element.tag == "node" or element.tag == "way":
        #print element.attrib
        #YOUR CODE HERE
        node["type"] = element.tag
        node["created"] = {}
        node["pos"] =[ None , None]
        
        for key,val in element.attrib.items():
            
            if key in CREATED:
                
                node["created"].update({key:val})
            elif key == 'lat':
                node["pos"][0] = float(element.get(key))
            elif key == 'lon':
                node["pos"][1] = float(element.get(key))
            else:
                node[key] = element.get(key)
                
                
        
        a= []
        
        for i in element:
            
            if i.tag == 'nd':
                
                for refs,val in i.attrib.items():
                    #print(val)
                    a.append(val)
            node["node_refs"] = a
            
            if i.tag == 'tag':
                
                node["address"] = {}
                empty_dict = {}
               
                for loc,val in i.attrib.items():
                    
                    if val.startswith("addr:") and lower_colon.match(val):
                                
                                key, value = val.split(":")
                                #empty_dict = dict.fromkeys([value])
                                new_val = update_name(i.attrib['v'], mapping)
                                
                                if value == 'city':
                                    if new_val.lower() != 'mumbai':
                                        address[value] = 'Mumbai'
                                        address["location"] = new_val
                                else:
                                    address[value] = new_val
                                
                    elif "addr:" not in val and lower.match(val) and loc != 'v':
                        if val.lower() == "phone" or val.lower() == 'fax':
                            node[val] = update_telephone_no(i.attrib['v'])
                            
                        elif val.lower() == "cuisine":
                            node[val] = update_food(i.attrib['v'])
                        elif val.lower() == "amenity":
                            if 'atm' in i.attrib['v'].lower():
                                node[val] = i.attrib['v']
                                node["atm"] = "yes"
                            else:
                                node[val] = i.attrib['v']
                        else:
                            node[val] = i.attrib['v']
                    
                    elif "name" in val:
                        
                        if len(i.attrib['v']) != -1:
                            #print (i.attrib['v'])
                            new_name = update_name(i.attrib['v'], mapping)
                            node[val] = new_name
                            
                    elif problemchars.match(val):
                        pass
                                
                node["address"] = address   
                
        if "operator" in node and "name" not in node:
            node["name"] = node["operator"]
        
        if "brand" in node and "name" not in node:
            node["name"] = node["brand"]
            
        if "name" in node:
            if "atm" in node:
                #print(node["name"])
                node["name"] = update_bank_name(node["name"])  
                
        
        
    else:
        return None
    return node

def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return (data)

In [10]:
data = process_map('sample_mumbai.osm', True)


## Importing data to MongoDB

For this purpose I am using the JSON file created above and running this command in the windows command prompt

#### mongoimport --db openStreetMapData --collection Mumbai --type json --file sample_mumbai.osm.json

## Connecting to MongoDB

In [11]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('localhost:27017')
db = client.openStreetMapData            #database name is openStreetMapData
coll = db.Mumbai                        # collection name is Mumbai



### Number of Total Documents (nodes and ways)

In [12]:
result = coll.find().count()
print(result)


232900


### Total number of Nodes in the document

In [13]:
coll.find( { "type": "node" } ).count()

204562

### Total number of Ways in the document

In [14]:
coll.find( { "type": "way" } ).count()

28298

### Number of unique contributers

In [15]:
len(coll.distinct( "created.user" ))

873

### User ID who has contributed the most

In [16]:
def pipeline():    
    pipeline = [{"$match" : {"created.user" : {"$exists" : True , "$ne" : None }}},                            
                
                {"$group": {"_id" : "$created.user",                
                            "count" : {"$sum" :1}}},
                                                
                {"$sort" : {"count" : -1}},
                
                {"$limit" : 1}
         
                ]
    
    return pipeline

output = pipeline()    
result = coll.aggregate(output)
for i in result:
    pprint(i)

{'_id': 'PlaneMad', 'count': 7852}


### List of top 10 amenities in the city

In [17]:
def pipeline():    
    pipeline = [{"$match" : {"amenity" : {"$exists" : True , "$ne" : None }}},                            
                
                {"$group": {"_id" : "$amenity",                
                            "count" : {"$sum" :1}}},
                                                
                {"$sort" : {"count" : -1}},
                
                {"$limit" : 10}
         
                ]
    
    return pipeline

output = pipeline()    
result = coll.aggregate(output)
for i in result:
    pprint(i)

{'_id': 'place_of_worship', 'count': 53}
{'_id': 'restaurant', 'count': 41}
{'_id': 'bank', 'count': 30}
{'_id': 'school', 'count': 29}
{'_id': 'cafe', 'count': 19}
{'_id': 'hospital', 'count': 18}
{'_id': 'atm', 'count': 17}
{'_id': 'fast_food', 'count': 17}
{'_id': 'parking', 'count': 15}
{'_id': 'fuel', 'count': 14}


Since India is a country which has many religions there must be many places of worship for all those religions. So i guess the output is justifying it.

### How many fuel stations are there in the city

In [18]:
coll.find( { "amenity" : "fuel" }).count()

14

### Number of atms in sample location of Mumbai I have selected

In [19]:
result = coll.find({"atm" : "yes"}).count()
print(result)


36


### Which bank has the most ATMs

In [20]:
def pipeline():    
    pipeline = [{"$match" : {"name" : {"$exists" : True , "$ne" : None },
                            "atm" : "yes"}},
                
                {"$group": {"_id" : "$name",                
                            "count" : {"$sum" :1}}},
                
                {"$project": {"name" : "$name",
                             "number_of_atms": "$count"}},
                
                {"$sort" : {"number_of_atms" : -1}}
         
                ]
    
    return pipeline

output = pipeline()    
result = coll.aggregate(output)
for i in result:
    pprint(i)

{'_id': 'HDFC Bank', 'number_of_atms': 5}
{'_id': 'State Bank of India', 'number_of_atms': 5}
{'_id': 'Union Bank of India ', 'number_of_atms': 3}
{'_id': 'ICICI Bank', 'number_of_atms': 2}
{'_id': 'Kotak Mahindra Bank', 'number_of_atms': 2}
{'_id': 'Raheja Classique Bank', 'number_of_atms': 1}
{'_id': 'Karnataka Bank', 'number_of_atms': 1}
{'_id': 'private', 'number_of_atms': 1}
{'_id': 'Corporation Bank', 'number_of_atms': 1}
{'_id': 'THE HINDUSTHAN CO-OP.BANK LTD MUMBAI', 'number_of_atms': 1}
{'_id': 'Dena Bank', 'number_of_atms': 1}
{'_id': 'AXIS Bank', 'number_of_atms': 1}
{'_id': 'Punjab National Bank', 'number_of_atms': 1}
{'_id': 'IDBI', 'number_of_atms': 1}
{'_id': 'Bank of India', 'number_of_atms': 1}
{'_id': 'HSBC Bank', 'number_of_atms': 1}
{'_id': 'Greater Bombay Co-operative Bank', 'number_of_atms': 1}
{'_id': 'DBS Bank', 'number_of_atms': 1}


We can see that HDFC bank and State Bank of India has the most number of ATMs

### Most popular bank in the region

In [21]:
def pipeline():    
    pipeline = [{"$match" : {"amenity" :{ "$eq" : "bank", "$ne" : None},
                            "name" : {"$exists" : True , "$ne" : None }}},
                
                {"$group": {"_id" : "$name",
                
                            "count" : {"$sum" :1}}},
                
                {"$project": {"name" : "$name",
                             "popularity_score": "$count"}},
                
                {"$sort" : {"popularity_score" : -1}}
         
                ]
    
    return pipeline

output = pipeline()    
result = coll.aggregate(output)
for i in result:
    pprint(i)


{'_id': 'State Bank of India', 'popularity_score': 4}
{'_id': 'HDFC Bank', 'popularity_score': 4}
{'_id': 'Kotak Mahindra Bank', 'popularity_score': 3}
{'_id': 'Union Bank of India ', 'popularity_score': 3}
{'_id': 'ICICI Bank', 'popularity_score': 2}
{'_id': 'Union Bank of India', 'popularity_score': 1}
{'_id': 'Karnataka Bank', 'popularity_score': 1}
{'_id': 'Greater Bombay Co-operative Bank', 'popularity_score': 1}
{'_id': 'THE HINDUSTHAN CO-OP.BANK LTD MUMBAI', 'popularity_score': 1}
{'_id': 'Dena Bank', 'popularity_score': 1}
{'_id': 'American Express Bank', 'popularity_score': 1}
{'_id': 'ANZ Grindlays', 'popularity_score': 1}
{'_id': 'Central Bank', 'popularity_score': 1}
{'_id': 'Bank of India', 'popularity_score': 1}
{'_id': 'ABN Amro', 'popularity_score': 1}
{'_id': 'Punjab National Bank', 'popularity_score': 1}
{'_id': 'Greater Bank', 'popularity_score': 1}


Here also I have got the result as expected most popular banks are HDFC Bank and State Bank of India

### Listing out the popular cuisines

In [22]:
def pipeline():    
    pipeline = [{"$match" : {"cuisine" :{ "$exists" : True, "$ne" : None}}},
                
                {"$project": { "cuisine": { "$toLower": "$cuisine" }}},
                            
                
                {"$group": {"_id" : "$cuisine",
                
                            "count" : {"$sum" :1}}},
                      
                
                {"$sort" : {"count" : -1}}
         
                ]
    
    return pipeline

output = pipeline()    
result = coll.aggregate(output)
for i in result:
    pprint(i)


{'_id': 'indian', 'count': 15}
{'_id': 'italian', 'count': 11}
{'_id': 'maharashtrian', 'count': 4}
{'_id': 'american', 'count': 2}
{'_id': 'continental', 'count': 2}
{'_id': 'chinese', 'count': 2}
{'_id': 'seafood', 'count': 1}


So we can see that Indian and Italian cuisine are more preferred in Mumbai. 

## Additional Improvements in Dataset

Earlier while querying the data, I saw there were numerous documents which did not have any name field , any address or any proper attribute which would highlight what the data is about. In MongoDB I have inserted those kind of documents, since it is not feasible to cleanse such kind of data. There can be thousands of such datasets where you don't have any proper highlighting key. So it be a tedious job to cleaning out all these uninformative elements. But we need to find a way to ignore these kind of data since these takes up much of the space. I hope to contribute to this by searching the internet and finding new ways to do this.

So now I find the number of documents that do not include name fields in them.

In [28]:
coll.find({"name" : {"$exists" : False }}).count()

231167

SO earlier we got total number of documents as 232900. SO we can see that only 1733 documents are perfectly formatted or we can say. About 99% of the data is filled with uninformative elements. So we need to find a way to remove them or they are taking up too many space. And this was just a sample what if there is a huge dataset. We need to find clever way to cleanse the data.

## Conclusion

The Mumbai data set was very messy with too much of bad data. I tried to cover as much of data wrangling and data cleaning as possible for our analysis and for this project. And for Open Street Map I think they should modify or update their process of taking inputs such that when a user is updating any location it should be mandatory to give a key which defines the location or else we would get bad data out of it. I hope the analysis covers some interesting insights about the data set.

## Bibliography

##### 1. https://wiki.openstreetmap.org/wiki/OSM_XML
##### 2. https://www.openstreetmap.org/#map=5/51.500/-0.100
##### 3. https://mapzen.com/data/metro-extracts/
##### 4. https://docs.mongodb.com/manual/
##### 5. https://stackoverflow.com/